In [ ]:
import sys
sys.path.insert(0, 'import file')
import model_import_file

path_to_data = "dataset"
path_to_save = "data_save/"

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import random
import tensorflow as tf
from keras.models import model_from_json
import json,codecs
from flask import Flask, render_template, request
from keras.preprocessing import image
import matplotlib.pyplot as plt
import cv2
import time
import random

def saveHist(path,history):
    
    new_hist = {}
    for key in list(history.history.keys()):
        new_hist[key]=history.history[key]
        if type(history.history[key]) == np.ndarray:
            new_hist[key] = history.history[key].tolist()
        elif type(history.history[key]) == list:
            if  type(history.history[key][0]) == np.float64:
                new_hist[key] = list(map(float, history.history[key]))
            
    print(new_hist)
    with codecs.open(path, 'w', encoding='utf-8') as file:
        json.dump(new_hist, file, separators=(',', ':'), sort_keys=True, indent=4) 

def loadHist(path):
    with codecs.open(path, 'r', encoding='utf-8') as file:
        n = json.loads(file.read())
    return n

class_dict = {}

counter = 0
for folder in os.listdir(path_to_data):
    path = os.path.join(path_to_data,folder)
    for subfolder in os.listdir(path):
        class_dict[subfolder] = counter
        counter += 1
    break

class_list = []
for ele in class_dict:
    class_list.append(ele)

IMG_SIZE_X = 200
IMG_SIZE_Y = 200
model_name = "model_1"

if os.path.exists(path_to_save + model_name + '.json'):
    json_file = open(path_to_save + model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model_1 = model_from_json(loaded_model_json)
    # load weights into new model
    model_1.load_weights(path_to_save + model_name + ".h5")
    print("Loaded model from disk")
    adagrad_opt = tf.keras.optimizers.Adagrad(
        learning_rate=0.0005,
        initial_accumulator_value=0.1,
        epsilon=1e-07,
        name="Adagrad",
    )
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    model_1.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    # Load History
    model_1_history = loadHist(path_to_save + model_name + "_history.json")
else:  
    total_training_images_classwise = {}
    total_testing_images_classwise ={}
    
    X_train = []
    y_train = []

    X_test = []
    y_test = []

    for folder in os.listdir(path_to_data):
        path = os.path.join(path_to_data,folder)
        for subfolder in os.listdir(path):
            path_to_subfolder = os.path.join(path,subfolder)
            key = subfolder
            number_of_image_in_folder = len(os.listdir(path_to_subfolder))
            if folder == "validation":
                total_testing_images_classwise[subfolder] = number_of_image_in_folder
            else:
                total_training_images_classwise[subfolder] = number_of_image_in_folder
            for img in os.listdir(path_to_subfolder):
                try:
                    img_array = cv2.imread(os.path.join(path_to_subfolder,img))
                    if folder == "validation":
                        resized_array = cv2.resize(img_array, (IMG_SIZE_X,IMG_SIZE_Y))
                        X_test.append(resized_array)
                        y_test.append(class_dict[subfolder])
                    else:
                        resized_array = cv2.resize(img_array, (IMG_SIZE_X,IMG_SIZE_Y))
                        X_train.append(resized_array)
                        y_train.append(class_dict[subfolder])
                except Exception as e:
                    print("Exception is : "+e)
                    pass

    print("Size of X_train : " + str(len(X_train)))
    print("Size of y_train : " + str(len(y_train)))
    print("Size of X_test : " + str(len(X_test)))
    print("Size of y_test : " + str(len(y_test)))
    
    X_train_numpy_array = np.array(X_train)
    y_train = np.array(y_train)
    X_test_numpy_array = np.array(X_test)
    y_test = np.array(y_test)

    X_train_numpy_array = X_train_numpy_array/255
    X_test_numpy_array = X_test_numpy_array/255

    input_shape = IMG_SIZE_X, IMG_SIZE_Y, 3
    n_classes = counter
    epoch = 5
    # Adagrad
    adagrad_opt = tf.keras.optimizers.Adagrad(
        learning_rate=0.0005,
        initial_accumulator_value=0.1,
        epsilon=1e-07,
        name="Adagrad",
    )
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    
    model_1 = model_import_file.model(input_shape,n_classes)
    model_1.compile(optimizer=adagrad_opt,loss = loss , metrics=['accuracy'])
    model_1_history = model_1.fit(X_train_numpy_array,y_train,batch_size = 4,epochs=epoch)

    # Save Model
    model_json = model_1.to_json()
    with open(path_to_save + model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    model_1.save_weights(path_to_save + model_name + ".h5")
    # Save History
    saveHist(path_to_save + model_name + "_history.json",model_1_history)
    print("Saved model to disk")
    
def predict_label(img_path):
    img_path = os.path.normpath(img_path)
    img_array = cv2.imread(img_path)
    resized_array = cv2.resize(img_array, (IMG_SIZE_X,IMG_SIZE_Y))
    img_t =  resized_array.reshape(1,IMG_SIZE_X,IMG_SIZE_Y,3)
    y_pred = np.argmax(model_1.predict(img_t))
    
    return class_list[y_pred]

def take_picture():
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("test")
    img_counter = random.randrange(0, 10000, 3)
    take_one = 0;
    while True:
        ret, frame = cam.read()
        if not ret:
            print("failed to grab frame")
            break
        cv2.imshow("test", frame)
        
        time.sleep(2)
        img_name = "static/opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1
        take_one += 1

        if take_one ==1:
            break

    cam.release()
    cv2.destroyAllWindows()
    
    return img_name

app = Flask(__name__)

@app.route("/", methods=['GET', 'POST'])
def main():
    img_path = "static/start_image.png"
    return render_template("index.html",img_path = img_path)

@app.route("/about")
def about_page():
    return "Please subscribe  Artificial Intelligence Hub..!!!"

@app.route("/submit", methods = ['GET', 'POST'])
def get_output():
    if request.method == 'POST':
        img = request.files['my_image']
        img_path = "static/" + img.filename	
        img.save(img_path)

        p = predict_label(img_path)

    return render_template("index.html", prediction = p, img_path = img_path)

@app.route("/click_image",methods = ['GET','POST'])
def take_pic():
    if request.method == 'POST':
        img = take_picture()
        p = predict_label(img)
    return render_template("index.html", prediction = p, img_path = img)

if __name__ =='__main__':
    #app.debug = True
    app.run(debug = True, use_reloader=False)

Loaded model from disk
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/May/2022 17:19:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2022 17:19:31] "GET /static/start_image.png HTTP/1.1" 304 -
127.0.0.1 - - [10/May/2022 17:19:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [10/May/2022 17:20:16] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2022 17:20:16] "GET /static/img.png HTTP/1.1" 200 -


static/opencv_frame_4287.png written!


127.0.0.1 - - [10/May/2022 17:20:35] "POST /click_image HTTP/1.1" 200 -
127.0.0.1 - - [10/May/2022 17:20:35] "GET /static/opencv_frame_4287.png HTTP/1.1" 200 -
